In [1]:
# 1. Install Dependencies (if needed) 
#!pip install requests pandas geopandas mapboxgl shapely 

import requests

In [2]:
# 2. Setup 
MAPBOX_TOKEN = "pk.eyJ1IjoiZnJhbmNpc2NvcmljYXJkbyIsImEiOiJjbWNqOGo2NTIwMWVuMm1wejF2MjhxcjhhIn0.7Fr-AlCpx39doVOeX7mK9A" 
BASE_URL = "http://api:8000"

In [3]:
# 3. Request Aggregated Data 
params = { 
    "day": "Monday", 
    "period": "AM Peak" 
} 
 
response = requests.get(f"{BASE_URL}/aggregates/", params=params) 
geojson_data = response.json()

In [4]:
# 4. Visualize in Mapbox 
from mapboxgl.viz import ChoroplethViz 
from mapboxgl.utils import create_color_stops 
 
features = [ 
    { 
        "type": "Feature", 
        "geometry": f["geometry"], 
        "properties": { 
            "average_speed": f["average_speed"], 
            "road_name": f["road_name"] 
        } 
    } for f in geojson_data 
] 
 
viz = ChoroplethViz( 
    { 
        "type": "FeatureCollection", 
        "features": features 
    }, 
    access_token=MAPBOX_TOKEN, 
    color_property="average_speed", 
    color_stops=create_color_stops([10, 20, 30, 40, 50], 
colors="Reds"), 
    center=(-81.6557, 30.3322), 
    zoom=11, 
    line_width=1.5, 
    opacity=0.8
) 

# The `viz.show()` command triggers a UserWarning recommending the use of IPython.display.IFrame.
#viz.show()   

# This approach bypasses the warning by manually rendering the visualization with IFrame.
from IPython.display import IFrame

viz.create_html('map.html')
IFrame(src='map.html', width=900, height=600)



In [5]:
# 5. Optional: Tabular Summary 
import pandas as pd 
 
df = pd.DataFrame([ 
    { 
        "link_id": f["link_id"], 
        "avg_speed": f["average_speed"], 
        "road_name": f["road_name"], 
        "length": f["length"] 
    } for f in geojson_data 
]) 
 
df.sort_values("avg_speed").head(10)

,link_id,avg_speed,road_name,length
48685,1294644043,0.62,Deer Meadow Apartments,0.055302
20459,870384620,0.62,None,0.054059
24827,1051453610,0.62,Washburn Rd,0.025476
55914,1295793493,0.62,Kendall Lake Luxury Apartments,0.058409
11533,23058357,0.62,Lauderdale Dr E,0.085128
20428,864632946,0.62,Heartleaf Ct,0.060273
9328,23047237,0.62,Falling Leaf Trl,0.050331
24997,1088770968,0.62,Wagenhals Ln,0.209402
4522,23029187,0.62,Wahine Dr N,0.078914
5476,23032342,0.62,7th Ave N,0.118060
